<a href="https://colab.research.google.com/github/AhmedRezaShams/Python/blob/main/mobilenet_transferlearn_REZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive  #py framework
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client  import GoogleCredentials  #clint library
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='fabric_defect.zip'"}).GetList()[0]['id'] #is a method call in the Google Drive API for Python that lists all the files in the current user's Google Drive that match a certain query.
f = drive.CreateFile({'id': fid})
f.GetContentFile('fabric_defect.zip')
f.keys()
!unzip fabric_defect.zip


Archive:  fabric_defect.zip
   creating: Fabric defect/
   creating: Fabric defect/test/
   creating: Fabric defect/test/hole/
  inflating: Fabric defect/test/hole/1.jpg  
  inflating: Fabric defect/test/hole/10.jpg  
  inflating: Fabric defect/test/hole/10_processed (1).jpg  
  inflating: Fabric defect/test/hole/10_processed (2).jpg  
  inflating: Fabric defect/test/hole/10_processed (3).jpg  
  inflating: Fabric defect/test/hole/11.jpg  
  inflating: Fabric defect/test/hole/11_processed (1).jpg  
  inflating: Fabric defect/test/hole/11_processed (2).jpg  
  inflating: Fabric defect/test/hole/11_processed (3).jpg  
  inflating: Fabric defect/test/hole/12.jpg  
  inflating: Fabric defect/test/hole/12_processed (1).jpg  
  inflating: Fabric defect/test/hole/12_processed (2).jpg  
  inflating: Fabric defect/test/hole/12_processed (3).jpg  
  inflating: Fabric defect/test/hole/13.jpg  
  inflating: Fabric defect/test/hole/13_processed (1).jpg  
  inflating: Fabric defect/test/hole/13_proc

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121


In [ ]:
import cv2
import os
from tqdm import tqdm

folder_dir = '/content/Fabric defect/train'
SIZE = (224,224) 
DOWNSAMPLE_RATIO = 4 #image er resulution komai
JPEG_QUALITY = 60

# get the total number of files to process
total_files = sum(len(files) for _, _, files in os.walk(folder_dir))

# use tqdm to display a progress bar
with tqdm(total=total_files, desc="Processing Images") as pbar:
    for folder in os.listdir(folder_dir):
        for file in os.listdir(os.path.join(folder_dir, folder)):
            image_path = os.path.join(folder_dir, folder, file)
            # check if the file has read permission
            if not os.access(image_path, os.R_OK):
                print(f"No read permission for: {image_path}")
                continue
            # read the image
            img = cv2.imread(image_path)
            if img is None:
                print(f"Failed to read image: {image_path}")
                continue
            # check if the image is already in JPEG format
            if os.path.splitext(file)[-1].lower() in ['.jpg', '.jpeg']:
                img_resized = cv2.resize(img, SIZE)
                # write the resized image
                success = cv2.imwrite(image_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])
                if not success:
                    print(f"Failed to write image: {image_path}")
            else:
                # convert the image to JPEG format
                new_image_path = os.path.splitext(image_path)[0] + '.jpg'
                img_resized = cv2.resize(img, SIZE)
                # write the resized image in JPEG format
                success = cv2.imwrite(new_image_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])
                if not success:
                    print(f"Failed to write image: {new_image_path}")
                else:
                    # delete the original image and replace it with the new JPEG image
                    os.remove(image_path)
                    os.rename(new_image_path, image_path)
            pbar.update(1) # increment the progress bar


Processing Images: 100%|██████████| 666/666 [00:13<00:00, 50.40it/s] 


In [ ]:
gpus = tf.config.list_logical_devices('GPU') 
stg=tf.distribute.MirroredStrategy(gpus)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
# Add a new top (classification) layer for fabric classification
with stg.scope():
  base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base.layers:
    layer.trainable = False
  model = keras.Sequential([
      base,
      layers.Flatten(),
      layers.Dropout(0.5),
      layers.Dense(256, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(4, activation='softmax')
  ])


9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#Augmentation
train_data_dir = '/content/Fabric defect/train'
validation_data_dir = '/content/Fabric defect/validation'
batch_size = 32
img_height = 224
img_width = 224

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 666 images belonging to 4 classes.
Found 403 images belonging to 4 classes.


In [ ]:
# Train the model
from tensorflow.keras.optimizers.legacy import Adam
epochs = 30
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)


Epoch 1/30
20/20 [==============================] - 34s 1s/step - loss: 7.4049 - accuracy: 0.5789 - val_loss: 1.4651 - val_accuracy: 0.8177
Epoch 2/30
20/20 [==============================] - 23s 1s/step - loss: 1.7674 - accuracy: 0.7445 - val_loss: 0.4987 - val_accuracy: 0.8542
Epoch 3/30
20/20 [==============================] - 22s 1s/step - loss: 0.8110 - accuracy: 0.7681 - val_loss: 0.4045 - val_accuracy: 0.8542
Epoch 4/30
20/20 [==============================] - 22s 1s/step - loss: 0.5822 - accuracy: 0.7997 - val_loss: 0.3053 - val_accuracy: 0.8542
Epoch 5/30
20/20 [==============================] - 21s 1s/step - loss: 0.4485 - accuracy: 0.8186 - val_loss: 0.2796 - val_accuracy: 0.8620
Epoch 6/30
20/20 [==============================] - 29s 1s/step - loss: 0.4254 - accuracy: 0.8297 - val_loss: 0.3389 - val_accuracy: 0.8646
Epoch 7/30
20/20 [==============================] - 22s 1s/step - loss: 0.4410 - accuracy: 0.8375 - val_loss: 0.2776 - val_accuracy: 0.8880
Epoch 8/30
20/20 [==

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_generator)
print("Validation accuracy:", val_acc)

13/13 [==============================] - 15s 1s/step - loss: 0.1616 - accuracy: 0.9429
Validation accuracy: 0.9429280161857605


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# set the path to the validation directory
val_dir = '/content/Fabric defect/validation'

# create an ImageDataGenerator to preprocess the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# set batch size to 1 for testing
batch_size = 1

# use the validation directory to create a validation data generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# make predictions on the validation data
predictions = model.predict(val_generator, steps=val_generator.n)

# get the actual labels from the validation data generator
y_true = val_generator.classes

# get the predicted labels from the model predictions
y_pred = np.argmax(predictions, axis=1)

# get the class labels from the validation data generator
class_labels = list(val_generator.class_indices.keys())

# print the actual and predicted classes for the first 15 images
acc = 0
for i in range(100):
    actual_class = class_labels[y_true[i]]
    predicted_class = class_labels[y_pred[i]]
    if actual_class == predicted_class:
      acc+=1
    print(f"Actual class: {actual_class}, Predicted class: {predicted_class}")

print("Acc:",acc,"%")


Found 403 images belonging to 4 classes.
403/403 [==============================] - 14s 29ms/step
Actual class: Lines, Predicted class: hole
Actual class: Lines, Predicted class: hole
Actual class: Lines, Predicted class: hole
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Predicted class: Lines
Actual class: Lines, Pred